# Install required libraries for solution 

In [16]:
!pip install pickle5

In [ ]:
!pip install cloudpickle==2.1.0

In [ ]:
!pip install gym==0.21.0

In [ ]:
!pip install gym-retro

In [ ]:
!pip install opencv-contrib-python --user

In [ ]:
!pip install matplotlib

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
!pip install stable-baselines3[extra] optuna

In [ ]:
!pip install protobuf==3.20.1 --user

In [1]:
!pip install tensorflow

## Import all required libraries

In [1]:
import pickle5 as pickle
# Import retro to play Street Fighter using a ROM
import retro
# Import time to slow down game
import time

# Import environment base class for a wrapper 
from gym import Env 
# Import the space shapes for the environment
from gym.spaces import MultiBinary, Box
# Import numpy to calculate frame delta 
import numpy as np
# Import opencv for grayscaling
import cv2
# Import matplotlib for plotting the image
from matplotlib import pyplot as plt



# Importing the optimzation frame - HPO
import optuna
# Import os to deal with filepaths
import os
# PPO algo for RL
from stable_baselines3 import PPO
# Bring in the eval policy method for metric calculation
from stable_baselines3.common.evaluation import evaluate_policy
# Import the sb3 monitor for logging 
from stable_baselines3.common.monitor import Monitor
# Import the vec wrappers to vectorize and frame stack
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3 import DQN
from typing import Any, Dict

# Import base callback 
from stable_baselines3.common.callbacks import BaseCallback

from gym import spaces
import gym

# used for creating date in required formate
from datetime import date

#To be able to log custom scalars
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

## Setup paths to log , record agent game play

In [2]:
LOG_DIR = './logs_final/final_results_statical_dnq/'
OPT_DIR = './opt/final_results_statical_ppo/'
CHECKPOINT_DIR = './train/final_results_statical_dnq/'
model_name = 'final_results_statical_dnq_' + date.today().strftime('%Y-%m-%d')
RECORD_PATH= './RecordAgentsGamePlay/final_results_statical_dnq'

# Create game environment wrapper

In [3]:
#Adapted class called StreetFighter from https://github.com/nicknochnack/StreetFighterRL/blob/main/StreetFighter-Tutorial.ipynb
#Made changes to the __init__ function to allow the user the option to record the game play , changed it work for Alien Soldier
#enviroment , added logic to set buttons and game state name.
#Made changes to reset function added own custom attributes which needed to be reset once game restarts  
#Made changes to step function added logic which sets penalties and to update custom attributes created.

# Create custom environment 
class AlienSoldier(Env): 
    def __init__(self,record_results=False,record_path=''):
        super().__init__()
        
        # Specify action space and observation space 
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space =  spaces.MultiBinary(12)
        # Set the buttons as they needed when wrapped with AlienSoldierDiscretizer
        self.buttons = ['B', 'A', 'MODE', 'START', 'UP', 'DOWN', 'LEFT', 'RIGHT', 'C', 'Y', 'X', 'Z']
    
        # Startup and instance of the game 
        if record_results:
            self.game = retro.make(game='AlienSoldier-Genesis', use_restricted_actions=retro.Actions.FILTERED, record=record_path)
        else:
            self.game = retro.make(game='AlienSoldier-Genesis', use_restricted_actions=retro.Actions.FILTERED)
            
        #set the game state name so able to access which level the game is on
        self.statename = self.game.statename
    
    def reset(self):
        # Return the first frame 
        obs = self.game.reset()
        obs = self.preprocess(obs) 
        self.previous_frame = obs 
        self.statename = self.game.statename
        # reset attribute which holds the score delta , health , time for next game 
        self.score = 0 
        self.health = 512
        self.time = 157
        
        return obs
    
    def preprocess(self, observation): 
        # Convert observation to gray scale 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resize the frame
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        # Add the channels value
        channels = np.reshape(resize, (84,84,1))
        return channels 
    
    def step(self, action): 
        
        # Take a step 
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs) 
        
        # Frame delta 
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs 
        
        # Reshape the reward function
        reward = info['score'] - self.score 
        
        # Add in penalties 
        if self.health < info['health'] :
            reward = - 10
        
        if self.health > info['health'] :
            reward = 20
        
        if self.time != info['time']:
            reward = -5
            
        # update custom attributes 
        self.health = info['health']
        self.time =  info['time'] 
        self.score = info['score'] 
        
        return frame_delta, reward, done, info
    
    def render(self, *args, **kwargs):
        self.game.render()
        
    def close(self):
        self.game.close()

## Build wrapper for Gym Env so actions space is combo's

In [4]:
#Code not mine taken from https://github.com/openai/retro/blob/master/retro/examples/discretizer.py 
#Made be called AlienSoldierDiscretizer instead of SonicDiscretizer , also changed the combo list added in the class to 
#be one that could be used in Alien Soldier

class Discretizer(gym.ActionWrapper):
    """
    Wrap a gym environment and make it use discrete actions.
    Args:
        combos: ordered list of lists of valid button combinations
    """

    def __init__(self, env, combos):
        super().__init__(env)
        assert isinstance(env.action_space, gym.spaces.MultiBinary)
        buttons = env.unwrapped.buttons
        self._decode_discrete_action = []
        for combo in combos:
            arr = np.array([False] * env.action_space.n)
            for button in combo:
                arr[buttons.index(button)] = True
            self._decode_discrete_action.append(arr)

        self.action_space = gym.spaces.Discrete(len(self._decode_discrete_action))

    def action(self, act):
        return self._decode_discrete_action[act].copy()


class AlienSoldierDiscretizer(Discretizer):
    """
    based on https://github.com/openai/retro-baselines/blob/master/agents/sonic_util.py
    """
    
    def __init__(self, env):
        # Added own combo list
        super().__init__(env=env, combos=[['LEFT'], 
                                          ['RIGHT'],
                                          ['C'],
                                          ['C','LEFT'],
                                          ['C','RIGHT'],
                                          ['B'],
                                          ['B','B'],
                                          ['LEFT','B','B'],
                                          ['RIGHT','B','B']
                                         ])

## Helper functions for hyperparameter tuning

In [5]:
# Taken from https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/utils/hyperparams_opt.py
# Made a adjustment to buffer_size so within range lapetop could handel
# Return test hyperparameters for DNQ model
def optimize_dqn(trial: optuna.Trial) -> Dict[str, Any]:
    """
    Sampler for DQN hyperparams.
    :param trial:
    :return:
    """
    gamma = trial.suggest_categorical("gamma", [0.9, 0.95, 0.98, 0.99, 0.995, 0.999, 0.9999])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 100, 128, 256, 512])
    buffer_size = trial.suggest_categorical("buffer_size", [int(1e4), int(2e4), int(3e4),int(5e4)])
    exploration_final_eps = trial.suggest_uniform("exploration_final_eps", 0, 0.2)
    exploration_fraction = trial.suggest_uniform("exploration_fraction", 0, 0.5)
    target_update_interval = trial.suggest_categorical("target_update_interval", [1, 1000, 5000, 10000, 15000, 20000])
    learning_starts = trial.suggest_categorical("learning_starts", [0, 1000, 5000, 10000, 20000])

    train_freq = trial.suggest_categorical("train_freq", [1, 4, 8, 16, 128, 256, 1000])
    subsample_steps = trial.suggest_categorical("subsample_steps", [1, 2, 4, 8])
    gradient_steps = max(train_freq // subsample_steps, 1)

    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small", "medium"])

    net_arch = {"tiny": [64], "small": [64, 64], "medium": [256, 256]}[net_arch]

    hyperparams = {
        "gamma": gamma,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "buffer_size": buffer_size,
        "train_freq": train_freq,
        "gradient_steps": gradient_steps,
        "exploration_fraction": exploration_fraction,
        "exploration_final_eps": exploration_final_eps,
        "target_update_interval": target_update_interval,
        "learning_starts": learning_starts,
        "policy_kwargs": dict(net_arch=net_arch),
    }

    return hyperparams

In [6]:
#Taken from https://github.com/nicknochnack/StreetFighterRL/blob/main/StreetFighter-Tutorial.ipynb
# Return test hyperparameters for PPO model
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [7]:
#Adapted from https://github.com/nicknochnack/StreetFighterRL/blob/main/StreetFighter-Tutorial.ipynb
# Runs a trail and returns the mean reward 
def optimize_agent(trial,model_algorithm):
    try: 

        # Create environment 
        env = AlienSoldier()
        
        # Depending on algorithm will need to call a different function to get trial params
        if model_algorithm == "dnq":
            
            model_params = optimize_dqn(trial)
            #The DNQ needs the action space to be Discrete but AlienSoldier has it as MultiBinary so we need wrap it 
            # with AlienSoldierDiscretizer which makes the action space Discrete
            env = AlienSoldierDiscretizer(env)
        else:
            model_params = optimize_ppo(trial)
        
        # Wrap the env with wrappers 
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Create model with required algorithm
        if model_algorithm == "dnq":
            model = DQN("CnnPolicy", env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        else:
            model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
            
        #Train model for required num of steps
        model.learn(total_timesteps=100000)
#         model.learn(total_timesteps=100)
        
        # Evaluate model 
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        # Save the model for later use if user decides to use model as base 
        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        print("--------------------------")
        print("Error" , e)
        print("--------------------------")
        return -1000



## Set algorithm for model

In [8]:
model_algorithm="dnq"
# Taken from https://www.kaggle.com/general/261870
# Wrap the optimize_agent inside a lambda reason need to pass more args to optimize_agent
func_optimize_agent = lambda trial: optimize_agent(trial,model_algorithm)

## Find the best hyperparameter for model

In [9]:
# Creating the experiment 
study = optuna.create_study(direction='maximize')
study.optimize(func_optimize_agent, n_trials=100, n_jobs=1 )

[I 2022-09-04 18:37:58,663] A new study created in memory with name: no-name-d27e3536-20a3-416d-b257-13e334d3da4e
C:\Users\Tiger\.conda\envs\game\lib\site-packages\stable_baselines3\ppo\ppo.py:141: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2541`, after every 39 untruncated mini-batches, there will be a truncated mini-batch of size 45
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2541 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2022-09-04 18:45:47,700] Trial 0 finished with value: 45.0 and parameters: {'n_steps': 2541, 'gamma': 0.9336208019550793, 'learning_rate': 2.4697426775067584e-05, 'clip_range': 0.1946541666231457, 'gae_lambda': 0.8627205620027374}. Best is trial 0 with value: 45.0.
C:\Users\Tiger\.conda\envs\game\lib\site-packages\stable_baselines3\ppo\ppo.py:141: UserWarning: You have specified a mini-batch size of 64, but becau

## Get best hyperparameters

In [10]:
study.best_params

{'n_steps': 3686,
 'gamma': 0.955177724605964,
 'learning_rate': 4.943397443677692e-05,
 'clip_range': 0.2599423012090678,
 'gae_lambda': 0.8508566377984575}

## Get best trial

In [11]:
study.best_trial

FrozenTrial(number=1, values=[75.0], datetime_start=datetime.datetime(2022, 9, 4, 18, 45, 47, 700424), datetime_complete=datetime.datetime(2022, 9, 4, 18, 53, 22, 979137), params={'n_steps': 3686, 'gamma': 0.955177724605964, 'learning_rate': 4.943397443677692e-05, 'clip_range': 0.2599423012090678, 'gae_lambda': 0.8508566377984575}, distributions={'n_steps': IntUniformDistribution(high=8192, low=2048, step=1), 'gamma': LogUniformDistribution(high=0.9999, low=0.8), 'learning_rate': LogUniformDistribution(high=0.0001, low=1e-05), 'clip_range': UniformDistribution(high=0.4, low=0.1), 'gae_lambda': UniformDistribution(high=0.99, low=0.8)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE, value=None)

## Best hyperparameters I used to trained for algorithm model

### DNQ 

```
model_params =  {'gamma': 0.99, 
                 'learning_rate': 0.0024514414607983074, 
                 'batch_size': 512, 
                 'buffer_size': 50000, 
                 'exploration_final_eps': 0.13882656073419855, 
                 'exploration_fraction': 0.0636702480159673, 
                 'target_update_interval': 20000, 
                 'learning_starts': 1000, 
                 'train_freq': 16
                 }
```

## PPO 
```
model_params = {'n_steps': 4153,
                'gamma': 0.8881436120036623,
                'learning_rate': 2.8578237422936424e-05,
                 'clip_range': 0.18402687390457662,
                 'gae_lambda': 0.8780780193937191
                }
```

## Setup callback function for logging to tensorboard_log for model

In [9]:
# Taken from https://github.com/nicknochnack/StreetFighterRL/blob/main/StreetFighter-Tutorial.ipynb
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [10]:
# Create the callback function to be used in the model tensorboard_log
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

# Train model 

### Create enviroment to train model

In [11]:
# Create environment 
env = AlienSoldier()

# Depending on algorithm may need to wrap it with AlienSoldierDiscretizer
if model_algorithm == "dnq":
    #The DNQ needs the action space to be Discrete but AlienSoldier has it as MultiBinary so we need wrap it 
    # with AlienSoldierDiscretizer which makes the action space Discrete
    env = AlienSoldierDiscretizer(env)

env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

### Setup model params 

In [11]:
#If your not going to load a model from hyperparameter tuning prcoess then you will need weights to use for model
#Below are params I used to train my model , users maybe be different according to best params 
#they get for there study on Optuna
if model_algorithm == "dnq":
    model_params =  {
                     'gamma': 0.99, 
                     'learning_rate': 0.0024514414607983074, 
                     'batch_size': 512, 'buffer_size': 50000, 
                     'exploration_final_eps': 0.13882656073419855, 
                     'exploration_fraction': 0.0636702480159673, 
                     'target_update_interval': 20000, 
                     'learning_starts': 1000, 
                     'train_freq': 16
                    }
else:
    #please note n_steps needs be divisible by 64 so that field may need to be updated on params taken from best trail 
    model_params = {'n_steps': 4153,
                     'gamma': 0.8881436120036623,
                     'learning_rate': 2.8578237422936424e-05,
                     'clip_range': 0.18402687390457662,
                     'gae_lambda': 0.8780780193937191
                   }
    #Making n_steps divisible by 64
    model_params['n_steps'] = 4160

### Create model for training 

In [16]:
if model_algorithm == "dnq":
    model =  DQN("CnnPolicy", env, tensorboard_log=LOG_DIR, verbose=1, **model_params)
else:
    model =  PPO("CnnPolicy", env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

Using cpu device
Wrapping the env in a VecTransposeImage.


C:\Users\Tiger\.conda\envs\game\lib\site-packages\stable_baselines3\common\buffers.py:221: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.82GB > 0.68GB
  "This system does not have apparently enough memory to store the complete "


### Load prevoius model to take advantage of previous weights 

In [17]:
#load previous model
#Pass in the best model here 
model.load(os.path.join(OPT_DIR, 'trial_1_best_model.zip'))

### Start training 

In [18]:
# Start training the model
model.learn(total_timesteps=5000000, callback=callback)

Logging to ./logs/statical_test_ppo/PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 3.22e+03 |
|    ep_rew_mean     | 206      |
| time/              |          |
|    fps             | 63       |
|    iterations      | 1        |
|    time_elapsed    | 58       |
|    total_timesteps | 3712     |
---------------------------------


## Evaluate the model 

In [47]:
#Update to the best model from training
if model_algorithm == "dnq":
    model = DQN.load(os.path.join(CHECKPOINT_DIR, 'best_model_10000000.zip'))
else:
    model = PPO.load(os.path.join(CHECKPOINT_DIR, 'best_model_7060000.zip'))


In [48]:
mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=1)

### Print the mean reward

In [46]:
mean_reward

1381.0

## Start agent playing game for required amount of games 

## Helper functions for agents

In [12]:
# A function to retrun the level from state name in the retro enviroment. The state name contains alot string data 
# around it e.g DefaultSettings.Level1.state only interested in the level int as would like to log which level agnet ends on
def getLevelFromStateName(state_name):
    
    # initializing substrings
    start_str = "evel"
    end_str = ".state"
 
    # getting index of substrings
    start_index = state_name.find(start_str)
    end_index = state_name.find(end_str)
    
    # sub string the level from state name as we only want level as number 
    level = state_name[start_index + len(start_str): end_index]

    # convert the level to int and retrun it 
    return int(level)

In [13]:
# Function to start game environment and have the statical agent play the number games passed to function  
def staticalAgentPlayGame(LOG_DIR,model_name,RECORD_PATH,RecordGamePlay,NumerOfGamesToPlay,
                          model_algorithm,model_to_Load):
        
    # Starts up the game environment
    
    # Create environment 
    env = AlienSoldier(record_results=RecordGamePlay,record_path=RECORD_PATH)

    # Depending on algorithm may need to wrap it with AlienSoldierDiscretizer
    if model_algorithm == "dnq":
        #The DNQ needs the action space to be Discrete but AlienSoldier has it as MultiBinary so we need wrap it 
        # with AlienSoldierDiscretizer which makes the action space Discrete
        env = AlienSoldierDiscretizer(env)

    env = Monitor(env, LOG_DIR)
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(env, 4, channels_order='last')
    
    # Create model
    if model_algorithm == "dnq":
        model =  DQN("CnnPolicy", env, tensorboard_log=LOG_DIR, verbose=1, **model_params)
#         model = DQN.load(os.path.join(CHECKPOINT_DIR, 'best_model_5000000.zip'))
        model = DQN.load(os.path.join(CHECKPOINT_DIR, model_to_Load))
    else:
        model =  PPO("CnnPolicy", env, tensorboard_log=LOG_DIR, verbose=1, **model_params)
#         model = PPO.load(os.path.join(CHECKPOINT_DIR, 'best_model_1000000.zip'))
        model = PPO.load(os.path.join(CHECKPOINT_DIR, model_to_Load))

    # Setup writer which will log the scalar values from end of agent game play 
    writer = tf.summary.create_file_writer(LOG_DIR,name=model_name)

    print("------------------------------------")
    # Reset game to starting state
    obs = env.reset()
    # Set done flag to flase this indicates if agent game is done
    done = False

    # Loop over number games to play   
    for game in range(NumerOfGamesToPlay): 
        print("Game Num" ,game + 1)

        #Check if agents game has ended 
        while not done: 
            # Render the game frame 
            env.render()
            # Predict the next action the agent should perform
            action = model.predict(obs)[0]
            
            
            #Set the values from step function
            obs, reward, done, info = env.step(action)    

            #Check if game is done, if true then proceed to log scalar values to writer to be logged 
            if done: 
                print("Game over")
                print("states",info[0])
                
    
                # Log the values to writer 
                with writer.as_default():
                    tf.summary.scalar("score", info[0]["score"], step=game + 1)
                    tf.summary.scalar("time", info[0]["time"], step=game + 1)
                    tf.summary.scalar("health", info[0]["health"], step=game + 1)
                    tf.summary.scalar("level", getLevelFromStateName(env.get_attr("statename")[0]), step=game + 1)
                    writer.flush()
                print("------------------------------------")
        # Reset obs so game can restart
        obs = env.reset()
        done = False
    env.close()


## Start game enviroment and start agent playing game for required amount of games 

In [14]:
# If you get a error of Cannot create multiple emulator instances per process,please run last code block in solution
staticalAgentPlayGame(LOG_DIR,model_name,RECORD_PATH,True,1,model_algorithm,'best_model_5000000.zip')

Using cpu device
Wrapping the env in a VecTransposeImage.


C:\Users\Tiger\.conda\envs\game\lib\site-packages\stable_baselines3\common\buffers.py:221: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.82GB > 0.55GB
  "This system does not have apparently enough memory to store the complete "
C:\Users\Tiger\.conda\envs\game\lib\site-packages\stable_baselines3\common\buffers.py:221: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 0.56GB > 0.49GB
  "This system does not have apparently enough memory to store the complete "


------------------------------------
Game Num 1


C:\Users\Tiger\.conda\envs\game\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


Game over
states {'health': 0, 'score': 3063, 'time': 115, 'episode': {'r': 583, 'l': 2598, 't': 227.963948}, 'terminal_observation': array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       ...,

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0,

## Convert game play to mp4

Update the below cmd to path where your game play is saved 

In [ ]:
%run -m retro.scripts.playback_movie RecordAgentsGamePlay/final_results_statical_dnq/AlienSoldier-Genesis-DefaultSettings.Level1-000001.bk2

## Run below block if get error of Cannot create multiple emulator instances per process

In [28]:
env.close()

##  End